# ASU HRT Workshop - Landlab Use Case Part 2

## Introduction

Overland flow, particularly the infiltration-excess mechanism, is affected by the nature of water input through precipitation. This Jupyter notebook demonstrates how to use high resolution topography data and Landlab model components to delineate a watershed and simulate overland flow for a study area in Boulder County. 

This notebook includes the following sections:
- [Initial Setup](#setup)
  
  Install Python packages and create the output folder.
  <br>
- [Step 1: OpenTopography Data](#step1) 

  Load the topography datasets with 30m resolution and 1m resolution for the study area.
  <br>
- [Step 2: Delineate Watershed](#step2)

  Use the topography dataset (1m resolution) and Landlab components to delineate the watershed for the study area. 
  <br>
- [Step 3: Calculate Overland Flow](#step3)

  Use the watershed topography data and Landlab overlandflow component to simulate the surface water depth and discharge.
  <br>
- [Step 4: Visualize Results](#step4)

  Visualize the final results as a short video.


<a id='setup'></a>
## Initial Setup

### Install Packages (on Local PC)
If you want to run this notebook on your PC, you can run the following command which will create a separate conda environment named "overland_flow" and install all the required packages for you. After the installation, please make sure to activate the environment to run this notebook.

In [ ]:
# ! conda env create --file=environment.yml

### Create Folder
We will first import all the python packages and create a result folder for this notebook.

In [ ]:
import os
import warnings
from pathlib import Path

import numpy as np
import imageio.v2 as imageio
import matplotlib.pyplot as plt
from IPython.display import Video
from tqdm import trange

from bmi_geotiff import GeoTiff
from bmi_topography import Topography
from landlab import RasterModelGrid, imshow_grid
from landlab.components import FlowAccumulator, ChannelProfiler, OverlandFlow
from landlab.utils import get_watershed_mask

warnings.simplefilter(action="ignore", category=FutureWarning)

In [ ]:
results_dir = Path.cwd() / "results_betasso"
results_dir.mkdir(exist_ok=True)

print(results_dir)

<a id='step1'></a>
## Step 1 OpenTopography Data

### Study Area
Betasso is situated in lower Boulder Canyon, just 6 miles west of the City of Boulder. This area is a mix of steep forested slopes with several intermittent streams, and sloping meadows, sub-summits, and rock outcrops. The south facing slopes are primarily warmer with Ponderosa Pine stands. The cooler north-facing slopes are a Ponderosa Pine-Douglas fir mix. The Boulder Filtration Plant is located just outside the Betasso Preserve. For more details please check [here](https://czo-archive.criticalzone.org/boulder/infrastructure/field-area/betasso/).

<img src="https://czo-archive.criticalzone.org/images/made/images/national/photos-and-images/Boulder/Betasso/IMGP1431_640_480_80auto.jpg" width="800">

### 30m DEM

We will first use the CSDMS [Topography data component](https://csdms.colorado.edu/wiki/Model:Topography_Data_Component) to download the 30m DEM for the study area from [OpenTopography](https://opentopography.org/developers) and get its metadata and data values. 

This subsection demonstrates how to use the CSDMS data component to access the global topography data from OpenTopography. In the next subsection, we will load the 1m topography dataset for watershed delineation. 

In [ ]:
dem_30m = Topography(
    dem_type="SRTMGL1",
    south=40.0073,
    north=40.0169,
    west=-105.3445,
    east=-105.3327,
    output_format="GTiff",
    cache_dir=results_dir,
)
dem_30m.load()

**``HTTPError``? Click details for the solution.**

<details>

If you get ``HTTPError: 401 Client Error``, you will need to create an API key file to download the dataset. Please make sure you first request an [Open Topography API Key](https://opentopography.org/blog/introducing-api-keys-access-opentopography-global-datasets). 
    
Then copy and paste the following code in a new cell and run it. The install_api_key( ) function will ask for your Open Topography API Key to create the API key file for you. Please make sure you provide the correct API Key information. 

```python
from utils import install_api_key
install_api_key()
```
</details>

In [ ]:
dem_30m.da

In [ ]:
# plot DEM data
plt.imshow(dem_30m.da.values.squeeze())

plt.title("Topography Data (30m resolution)")
plt.xlabel("X (grid index)")
plt.ylabel("Y (grid index)")
plt.ticklabel_format(useOffset=False)
plt.colorbar(label="Elevation (m)")

### 1m DEM

For watershed delineation, we will use the 1m topography dataset as the input. This dataset is a subset from the data product of [Boulder Creek Critical Zone Observatory August 2010 LiDAR Survey](https://portal.opentopography.org/datasetMetadata?otCollectionID=OT.032012.26913.1). We have already downloaded this dataset and will use the [GeoTiff data component](https://csdms.colorado.edu/wiki/Model:GeoTiff_Data_Component) to load the dataset. 

In [ ]:
# Load topography data
dem_1m = GeoTiff("betasso_1m.tif")

In [ ]:
dem_1m.da

In [ ]:
# plot DEM data
plt.imshow(dem_1m.da)
plt.title("Topography Data (1m resolution)")
plt.xlabel("X (m)")
plt.ylabel("Y (m)")
plt.colorbar(label="elevation(m)")

<a id='step2'></a>
## Step 2 Delineate Watershed

We will use the 1m topography dataset for watershed delineation. The Landlab components ([FlowAccumulator](https://landlab.readthedocs.io/en/master/reference/components/flow_accum.html) and [ChannelProfiler](https://landlab.readthedocs.io/en/master/reference/components/channel_profiler.html)) and the Landlab utility ([get_watershed_mask](https://landlab.readthedocs.io/en/master/reference/utils/watershed.html)) will be used to accomplish this task. 

The general steps for watershed delineation include: 
- Setup raster model grid and add topographic elevation data field
- Calculate the flow accumulation
- Get watershed mask
- Set watershed boundary 

In [ ]:
# set the grid resolution (m)
grid_resolution = 1.0

# set up raster model grid
model_grid = RasterModelGrid(dem_1m.da.shape, xy_spacing=grid_resolution)

# add topographic elevation data field
model_grid.add_field("topographic__elevation", dem_1m.da.astype("float"))

# plot the topographic elevation data field
plt.figure(figsize=(10, 5))
imshow_grid(
    model_grid,
    "topographic__elevation",
    plot_name="Data field of topographic elevation",
    var_name="Elevation (m)",
)
plt.xlabel("X (m)")
plt.ylabel("Y (m)")
plt.gca().invert_yaxis()

In [ ]:
# calculate the flow accumulation
fa = FlowAccumulator(
    model_grid,
    method="Steepest",
    flow_director="FlowDirectorSteepest",
    depression_finder="LakeMapperBarnes",
    redirect_flow_steepest_descent=True,
    reaccumulate_flow=True,
)
fa.run_one_step()

In [ ]:
# plot the flow accumulation result
plt.figure(figsize=(10, 5))
imshow_grid(
    model_grid,
    np.sqrt(model_grid.at_node["drainage_area"]),
    plot_name="Data field of drainage area",
    var_name="square root of drainage area",
)
plt.xlabel("X (m)")
plt.ylabel("Y (m)")
plt.gca().invert_yaxis()

In [ ]:
# set up channel profiler
profiler = ChannelProfiler(model_grid, number_of_watersheds=1)
profiler.run_one_step()

In [ ]:
# get watershed mask
outlet = profiler.nodes[0][0]
watershed_mask = get_watershed_mask(model_grid, outlet)

In [ ]:
# set watershed boundary
model_grid.at_node["topographic__elevation"][~watershed_mask] = -9999.0
model_grid.status_at_node[~watershed_mask] = model_grid.BC_NODE_IS_CLOSED
model_grid.status_at_node[outlet] = model_grid.BC_NODE_IS_FIXED_VALUE

In [ ]:
# plot the watershed
plt.figure(figsize=(10, 5))

imshow_grid(
    model_grid,
    "topographic__elevation",
    plot_name="Topographic elevation of Betasso watershed",
    colorbar_label="Elevation (m)",
)
plt.xlabel("X (m)")
plt.ylabel("Y (m)")
plt.gca().invert_yaxis()

<a id='step3'></a>
## Step 3  Calculate Overland Flow

In this step, the watershed topography dataset and the Landlab [Overland Flow component](https://landlab.readthedocs.io/en/master/user_guide/overland_flow_user_guide.html#background-on-overlandflow-component) will be used for the simulation. The model run time is set as 5 min with the first 2 min assigned with a constant rainfall intensity (100 mm/hr). This rain intensity value is based on the observational data from an overland flow event that happened in Betasso watershed on Aug 30th, 2016. The simulation results will create a discharge time series plot at the outlet and a 2D plot of the surface water depth at each 60 sec time step.

In this simulation, we suppose there is no water infiltration process involved. Is this justified? Not really, this is a model simplification for demonstration purposes. So this assumption provides an end-member case of extreme soil water repellency in the watershed.

In [ ]:
# add surface wave depth data field
# set initial surface water depth value
model_grid.add_full("surface_water__depth", 1e-12, at="node", clobber=True)

plt.figure(figsize=(10, 5))
imshow_grid(
    model_grid,
    "surface_water__depth",
    plot_name="Data field of surface water depth",
    var_name="water depth (m)",
    cmap="Blues",
    vmin=0.0,
    vmax=1,
)
plt.xlabel("X (m)")
plt.ylabel("Y (m)")
plt.gca().invert_yaxis()

In [ ]:
# instantiate overland flow component
overland_flow = OverlandFlow(model_grid, steep_slopes=True)

# set model run parameters
elapsed_time = 0.0
model_run_time = 5 * 60  # duration of run (s)
storm_duration = 2 * 60  # duration of rain (s)
time_step = 60
rainfall_intensity = 100 / (1000 * 3600)  # mm/hr to m/s

outlet_discharge = []
outlet_times = []

# run overland flow simulation
for time_slice in trange(time_step, model_run_time + time_step, time_step):

    while elapsed_time < time_slice:

        # get adaptive time step
        overland_flow.dt = min(overland_flow.calc_time_step(), time_slice)

        # set rainfall intensity
        if elapsed_time < (storm_duration):
            overland_flow.rainfall_intensity = rainfall_intensity
        else:
            overland_flow.rainfall_intensity = 0.0

        # run model
        overland_flow.overland_flow(dt=overland_flow.dt)

        # update elapsed time
        elapsed_time += overland_flow.dt

        # get discharge result
        discharge = overland_flow.discharge_mapper(
            model_grid.at_link["surface_water__discharge"], convert_to_volume=True
        )

        outlet_discharge.append(discharge[outlet])
        outlet_times.append(elapsed_time)

    # plot result
    fig, ax = plt.subplots(
        2, 1, figsize=(8, 9), gridspec_kw={"height_ratios": [1, 1.5]}
    )
    fig.suptitle("Results at {} min".format(time_slice / 60))

    ax[0].plot(outlet_times, outlet_discharge, "-")
    ax[0].set_xlabel("Time elapsed (s)")
    ax[0].set_ylabel("discharge (cms)")
    ax[0].set_title("Water discharge at the outlet")

    imshow_grid(
        model_grid,
        "surface_water__depth",
        cmap="Blues",
        vmin=0,
        vmax=0.5,
        var_name="surface water depth (m)",
    )
    ax[1].set_title("")
    ax[1].invert_yaxis()
    ax[1].set_xlabel("X (m)")
    ax[1].set_ylabel("Y (m)")

    plt.close(fig)
    fig.savefig(os.path.join(results_dir, "flow_{}.png".format(time_slice)))

print("Simulation is done")

<a id='step4'></a>
## Step 4 Visualize Results

Run the cell below and it will create a short video ("demo.mp4") based on the simulation results. 

In [ ]:
# Make a short video
img_files = [
    os.path.join(results_dir, file)
    for file in os.listdir(results_dir)
    if ".png" in file
]
img_files.sort(key=lambda x: os.path.getmtime(x))

video_file = os.path.join(results_dir, "demo.mp4")

with imageio.get_writer(
    os.path.join(video_file), mode="I", fps=1, macro_block_size=None
) as writer:
    for f in img_files:
        im = imageio.imread(os.path.join(results_dir, f))
        writer.append_data(im)

writer.close()

In this notebook, the model run time is set as 5 min. You can also reset it by changing the "model_run_time" variable with a larger value and run the cells in [Step 3](#step3) to see the whole process of the rainwater draining out of this watershed. The cell below shows a video of the simulation results with the model run time set as 20 min (model_run_time = 20 * 60). 

In [ ]:
# Display the video
video_file = os.path.join(results_dir, "overland_flow.mp4")
Video(video_file, embed=True, width=790, height=700)

## References
- Adams, J. M., Gasparini, N. M., Hobley, D. E. J., Tucker, G. E., Hutton, E. W. H., Nudurupati, S. S., and Istanbulluoglu, E.: The Landlab v1.0 OverlandFlow component: a Python tool for computing shallow-water flow across watersheds, Geosci. Model Dev., 10, 1645–1663, https://doi.org/10.5194/gmd-10-1645-2017, 2017.

- de Almeida, G. A., Bates, P., Freer, J. E., & Souvignet, M. (2012). Improving the stability of a simple formulation of the shallow water equations for 2‐D flood modeling. Water Resources Research, 48(5).
- Boulder Creek Critical Zone Observatory August 2010 LiDAR Survey. Distributed by OpenTopography. https://doi.org/10.5069/G93R0QR0 . Accessed: 2023-02-07